In [1]:
import train
import torch
import webdataset as wds
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import numpy as np

/home/jpinkney/miniconda3/envs/clip2latent/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from clip2latent.latent_prior import WPlusPriorNetwork

In [8]:
cfg = {"data": {
    'bs': 128,
    'format': 'webdataset',
    'path': 'data/webdataset/sg2-ffhq-1024-clip/{00000..99}.tar',
    'embed_noise_scale': 0.75,
    'sg_pkl': 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-1024x1024.pkl',
    'clip_variant': "ViT-B/32",
    'n_latents': 1,
    'latent_repeats': [18],
}
}
cfg = OmegaConf.create(cfg)
stats, data = train.load_data(cfg.data)

In [9]:
datum = next(data.__iter__())

In [10]:
datum[1].shape

torch.Size([128, 512])

In [6]:
net = WPlusPriorNetwork(dim=512, num_timesteps=100, depth=6, dim_head=64, heads=8)

In [7]:
device = "cuda:1"
t = torch.tensor([1, 2], dtype=torch.long, device=device)
net.to(device)
out = net(torch.randn(2, 18, 512, device=device),
          diffusion_timesteps=t,
          text_embed=torch.randn(2, 512, device=device))

In [8]:
prior = ZWPrior(net, image_embed_dim=512, timesteps=100, image_embed_scale=1.0)

NameError: name 'ZWPrior' is not defined

In [9]:
prior.to(device)
prior(
    image_embed=torch.randn(2, 18, 512, device=device), 
    text_embed=torch.randn(2, 512, device=device))

NameError: name 'prior' is not defined

In [11]:
d = datum[0].detach().cpu().numpy()
d.shape

(128, 512)

In [12]:
(np.sqrt(512)/datum[0].norm(dim=1)).mean()

tensor(2.1074, dtype=torch.float16)

In [36]:
x = datum[0]
scale = 0.75
orig_norm = x.norm(dim=-1, keepdim=True)
# print(x.norm(dim=1))

x = x/orig_norm

noise = torch.randn_like(x)
noise /= noise.norm(dim=-1, keepdim=True)
x += scale*noise

x /= x.norm(dim=-1, keepdim=True)
x *= orig_norm

print(x.norm(dim=1))

tensor([11.1016,  9.9531,  9.3594, 11.0391, 10.1094, 11.1328, 10.7031,  9.9062,
        10.4062, 11.2266, 11.5391, 11.3516, 10.7656, 10.9375, 11.5781, 11.3672,
        11.4531, 10.5469, 10.5938, 10.9453, 11.2031, 10.7266, 10.1172, 10.5234,
        11.6172, 10.8594, 10.0625, 10.1250, 10.4141, 11.0547,  9.3906, 10.8281,
        10.7500, 11.7500, 10.7812, 11.2109, 11.1172, 10.5234, 10.1797, 11.0781,
         9.6406, 10.0000, 10.7578, 10.4609, 10.3438, 11.7031, 10.9141, 11.1172,
        11.4922, 11.2344, 10.7812, 10.5938, 11.4062, 10.8906, 11.3438, 10.3359,
        10.8672, 11.7500, 11.8359, 12.3047, 10.4141, 11.9531, 10.4688, 10.2812,
        10.0312, 10.2031,  9.8984, 10.8906, 11.5156, 10.8906, 10.4688, 11.4531,
        11.5078, 11.3594, 10.8906, 11.2109, 10.4844, 10.9922, 11.3203, 11.0234,
        10.8906, 10.9297, 10.2266, 11.5391, 11.0078, 10.7734, 10.7734, 10.8750,
         9.8828, 10.0312, 11.3125, 11.2578, 11.4766, 10.3203,  9.9297, 11.3281,
        10.8359, 10.2109, 10.7578, 11.21

tensor(2.0918, dtype=torch.float16)

In [ ]:
np.linalg.norm(d[0,0].reshape(-1,1))

In [ ]:
d1, d2 = datum[0][0:4], datum[0][4:8]
torch.cosine_similarity(d1.float(), d2.float())

In [ ]:
stats["clip_features"][1][0].norm()

In [ ]:
1 - 0.9**2

In [ ]:
(0.19*torch.randn(512) + 0.9*torch.randn(512)).norm()

In [ ]:
(0.1**2 + 1**2)*orig_norm

In [ ]:
1/stats["clip_features"][1]